#### importing libraries

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

#### Loading train / test data

In [2]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

In [3]:
test_dataset = dsets.MNIST(root='./data',
                          train=False,
                          transform=transforms.ToTensor())

#### make dataset iterable

In [4]:
batch_size = 100
n_iters=3000
num_epochs = n_iters/(len(train_dataset)/batch_size)
num_epochs= int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

#### create model class

In [5]:
class FeedforwardNeuralnet(nn.Module):
    def __init__(self,input_size, hidden_size,num_classes):
        super(FeedforwardNeuralnet,self).__init__()
        #linear function
        self.fc1 = nn.Linear(input_dim,hidden_dim)
        
        self.sigmoid = nn.Sigmoid()
        
        self.fc2= nn.Linear(hidden_dim, output_dim)
        
    def forward(self,x):
        out = self.fc1(x)
        out= self.sigmoid(out)
        out = self.fc2(out)
        return out
    

#### instantiate model class

In [6]:
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralnet(input_dim,hidden_dim,output_dim)

#### instantiate loss class

In [8]:
criterion = nn.CrossEntropyLoss()

#### instantiate optimizer class

In [9]:
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

#### parameters

In [11]:
print(model.parameters())
print(len(list(model.parameters())))

print(list(model.parameters())[0].size())

print(list(model.parameters())[1].size())

print(list(model.parameters())[2].size())

print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x000001A734EF4888>
4
torch.Size([100, 784])
torch.Size([100])
torch.Size([10, 100])
torch.Size([10])


#### train model

In [18]:
iter = 0 
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1,28*28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        if iter % 250 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1,28*28))
                
                outputs=model(images)
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            accuracy = 100 * correct/total
            
            print('iterations:{}. Accuracy: {}'.format(iter,accuracy))

iterations:250. Accuracy: 92
iterations:500. Accuracy: 92
iterations:750. Accuracy: 92
iterations:1000. Accuracy: 92
iterations:1250. Accuracy: 92
iterations:1500. Accuracy: 92
iterations:1750. Accuracy: 93
iterations:2000. Accuracy: 93
iterations:2250. Accuracy: 93
iterations:2500. Accuracy: 93
iterations:2750. Accuracy: 93
iterations:3000. Accuracy: 93


# GPU model

In [19]:
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralnet(input_dim,hidden_dim,output_dim)
model.cuda()

FeedforwardNeuralnet(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [20]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [21]:
iter = 0 
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1,28*28).cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        if iter % 250 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1,28*28).cuda())
                
                outputs=model(images)
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted.cpu() == labels.cpu()).sum()
            accuracy = 100 * correct/total
            
            print('iterations:{}. Accuracy: {}'.format(iter,accuracy))

iterations:250. Accuracy: 77
iterations:500. Accuracy: 85
iterations:750. Accuracy: 88
iterations:1000. Accuracy: 89
iterations:1250. Accuracy: 90
iterations:1500. Accuracy: 90
iterations:1750. Accuracy: 90
iterations:2000. Accuracy: 91
iterations:2250. Accuracy: 91
iterations:2500. Accuracy: 91
iterations:2750. Accuracy: 91
iterations:3000. Accuracy: 92
